<a href="https://colab.research.google.com/github/zzyuzzz/123/blob/master/%E6%95%B0%E5%AD%97%E5%9B%BE%E5%83%8F%E5%A4%84%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import torch 
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Compose, Resize
from torchvision import datasets

In [67]:
training_data = datasets.WIDERFace(
    root='data',
    split='train',
    transform=ToTensor(),
    download=True
)

testing_data = datasets.WIDERFace(
    root='data',
    split='test',
    transform=ToTensor(),
    download=True
)

Files already downloaded and verified
Files already downloaded and verified


In [68]:
testing_data[0]

(tensor([[[0.0275, 0.0392, 0.0627,  ..., 0.0471, 0.0471, 0.0471],
          [0.0353, 0.0353, 0.0392,  ..., 0.0588, 0.0627, 0.0667],
          [0.0706, 0.0549, 0.0431,  ..., 0.0588, 0.0627, 0.0667],
          ...,
          [0.8667, 0.8118, 0.8510,  ..., 0.6667, 0.6941, 0.5804],
          [0.7961, 0.8471, 0.8627,  ..., 0.6353, 0.6745, 0.6000],
          [0.7843, 0.8824, 0.8667,  ..., 0.6118, 0.6902, 0.6353]],
 
         [[0.0549, 0.0667, 0.0902,  ..., 0.0392, 0.0392, 0.0392],
          [0.0627, 0.0627, 0.0667,  ..., 0.0510, 0.0549, 0.0588],
          [0.0980, 0.0824, 0.0706,  ..., 0.0510, 0.0549, 0.0588],
          ...,
          [1.0000, 0.9608, 0.9922,  ..., 0.5569, 0.5804, 0.4627],
          [0.9490, 1.0000, 1.0000,  ..., 0.5176, 0.5569, 0.4824],
          [0.9451, 1.0000, 1.0000,  ..., 0.4902, 0.5686, 0.5137]],
 
         [[0.0941, 0.1059, 0.1294,  ..., 0.0510, 0.0510, 0.0510],
          [0.1020, 0.1020, 0.1059,  ..., 0.0627, 0.0667, 0.0706],
          [0.1373, 0.1216, 0.1098,  ...,

In [69]:
batch_size = 1

train_dataloader = DataLoader(training_data, batch_size=batch_size,shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=batch_size)

In [70]:
i = 0
for X, y in train_dataloader:
  i = i + 1
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print("bbox", y['bbox'])
  if i > 10:
    break

Shape of X [N, C, H, W]: torch.Size([1, 3, 724, 1024])
bbox tensor([[[276, 218, 102, 134]]])
Shape of X [N, C, H, W]: torch.Size([1, 3, 701, 1024])
bbox tensor([[[138, 224, 162, 214],
         [252, 166, 220, 224]]])
Shape of X [N, C, H, W]: torch.Size([1, 3, 640, 1024])
bbox tensor([[[ 68,  14,  52,  68],
         [794,  40,  54,  64]]])
Shape of X [N, C, H, W]: torch.Size([1, 3, 681, 1024])
bbox tensor([[[248,  44,  78, 140],
         [366, 220,  60,  80],
         [542, 156,  66,  90],
         [690, 308,  60,  82],
         [810, 330,  58,  88]]])
Shape of X [N, C, H, W]: torch.Size([1, 3, 672, 1024])
bbox tensor([[[883, 138,  40,  32],
         [753, 141,  54,  41],
         [558, 124,  63,  51],
         [652, 145,  36,  31],
         [299, 137,  46,  42],
         [112, 122,  40,  37]]])
Shape of X [N, C, H, W]: torch.Size([1, 3, 1541, 1024])
bbox tensor([[[358, 343, 151, 202],
         [638, 907, 181, 199]]])
Shape of X [N, C, H, W]: torch.Size([1, 3, 472, 1024])
bbox tensor([[

In [71]:
class YOLO_V1(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, 7, 2),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d((2, 2), 2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 192, (3, 3)),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d((2, 2), 2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(192, 128, (1, 1)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(128, 256, (3, 3)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(256, 256, (1, 1)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, (3, 3)),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d((2, 2), 2)
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(512, 256, (1, 1)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, (3, 3)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, 256, (1, 1)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, (3, 3)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, 256, (1, 1)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, (3, 3)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, 256, (1, 1)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, (3, 3)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, 512, (1, 1)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, 1024, (3, 3)),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d((2, 2), 2)
        )
        self.layer5 = nn.Sequential(
            nn.Conv2d(1024, 512, (1, 1)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, 1024, (3, 3)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(1024, 512, (1, 1)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, 1024, (3, 3)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(1024, 1024, (3, 3), 2),
            nn.LeakyReLU(0.1),
        )
        self.layer6 = nn.Sequential(
            nn.Conv2d(1024, 1024, (3, 3)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(1024, 1024, (3, 3)),
            nn.LeakyReLU(0.1),
        )
        self.layer7 = nn.Sequential(
            nn.Conv2d(1024, 4096, (7, 7)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(4096, 245, (1, 1)),
            nn.LeakyReLU(0.1)
        )
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        print(x.shape)
        x = self.layer6(x)
        x = self.layer7(x)
        return torch.reshape(x, (7, 7, 5))


In [72]:
import math
lambda_coord = 5
lambda_noobj = .5

In [73]:
def squareDiff(x:tuple, y:tuple):
    t = x - y
    return t[0] ** 2 + t[1] ** 2

def objInCell(box, truth_box):
    center_x = truth_box[0] + truth_box[2] / 2
    center_y = truth_box[1] + truth_box[3] / 2
    return box[0] <= center_x and box[0] + box[2] >= center_x \
            and box[1] <= center_y and box[1] + box[3] >= center_y

In [74]:
S = 7

def Loss(X, bbox):
    loss_coord = 0
    loss_noobj = 0
    loss_0 = 0
    for x in X:
        for i in x.shape[0]:
            for j in x.shape[1]:
                center_x = x[i, j][0] - x[i, j][2] / 2
                center_y = x[i, j][1] - x[i, j][3] / 2
                for y in bbox:
                    if objInCell((center_x, 
                                center_y,
                                x[i, j][2],x[i, j][3]
                                ), y):
                        loss_coord = loss_coord + squareDiff((center_x, center_y), (y[0], y[1]))
                        loss_coord = loss_coord + squareDiff((math.sqrt(x[i, j][2]),
                                                            math.sqrt(x[i, j][3])), 
                                                            (math.sqrt(y[2]),
                                                            math.sqrt(y[3])))
                        loss_0 = loss_0 + (1 - x[i, j, 4]) ** 2
                    else:
                        loss_noobj = loss_noobj + x[i, j, 4] ** 2

    return lambda_coord * loss_coord + lambda_noobj * loss_noobj + loss_0



In [75]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [76]:
model = YOLO_V1().to(device)
print(model)

YOLO_V1(
  (layer1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2))
    (1): LeakyReLU(negative_slope=0.1)
    (2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.1)
    (2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(192, 128, kernel_size=(1, 1), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.1)
    (2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
    (3): LeakyReLU(negative_slope=0.1)
    (4): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (5): LeakyReLU(negative_slope=0.1)
    (6): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1))
    (7): LeakyReLU(negative_slope=0.1)
    (8): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer4): Sequential(
    (0): Conv2d(512,

In [77]:
for batch, (X, y) in enumerate(train_dataloader):
  print(y)
  break

{'bbox': tensor([[[244, 254,  71, 100],
         [366, 255,  66,  85],
         [442, 234,  74, 102],
         [867, 427,  20,  21],
         [779, 375,  18,  23],
         [678, 358,  13,  18],
         [610, 356,  15,  21],
         [592, 347,  11,  14],
         [957, 293,   9,  12],
         [883, 294,   8,   9]]]), 'blur': tensor([[1, 1, 1, 2, 1, 2, 1, 2, 2, 2]]), 'expression': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'illumination': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'occlusion': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'pose': tensor([[0, 0, 0, 1, 1, 1, 1, 1, 0, 0]]), 'invalid': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [78]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [79]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        y = y['bbox']
        X, y = X.to(device), y.to(device)
        print(X.shape)
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [80]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, Loss, optimizer)
    # test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
torch.Size([1, 3, 663, 1024])
torch.Size([1, 1024, 4, 10])


RuntimeError: ignored